# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador and API gateway**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

## Serve Single Model

In [ ]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set model.image.name=seldonio/mock_classifier_rest:1.2 --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!helm template ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-951ca8b

### Get predictions

In [1]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",oauth_key="oauth-key",oauth_secret="oauth-secret")

#### REST Request

In [2]:
p = sc.predict(gateway="seldon",transport="rest")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.6567095529913793
  }
}

Response:
meta {
  puid: "fkg67frfoi0sacdbuho2fk2adb"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_rest:1.2"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09449403845107879
  }
}



In [3]:
p = sc.predict(gateway="ambassador",transport="rest")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.04253433830734399
  }
}

Response:
meta {
  puid: "9d95giuu5j0ovg9pak1gujlpt2"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_rest:1.2"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05344720882863088
  }
}



#### gRPC Request

In [4]:
p = sc.predict(gateway="ambassador",transport="grpc")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.08337964070922887
  }
}

Response:
meta {
  puid: "h5b0fnvdukb5etetu8ajuns84b"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_rest:1.2"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05555168965647451
  }
}



In [5]:
p = sc.predict(gateway="seldon",transport="grpc")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.488271459086304
  }
}

Response:
meta {
  puid: "pa10a0fgkvctqjj3ajq1u71i3g"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_rest:1.2"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08103289026766958
  }
}



## Test Feedback

In [6]:
p = sc.predict(gateway="seldon",transport="rest")
f = sc.feedback(p.request,p.response,1.0,gateway="seldon",transport="rest")
print(f)

Success:True message:
Request:
request {
  data {
    tensor {
      shape: 1
      shape: 1
      values: 0.4724753550421932
    }
  }
}
response {
  meta {
    puid: "tkv75lsv0nog46evlokhohvqc0"
    requestPath {
      key: "classifier"
      value: "seldonio/mock_classifier_rest:1.2"
    }
  }
  data {
    names: "proba"
    tensor {
      shape: 1
      shape: 1
      values: 0.07986436636733006
    }
  }
}
reward: 1.0

Response:
None


In [7]:
p = sc.predict(gateway="ambassador",transport="rest")
f = sc.feedback(p.request,p.response,1.0,gateway="ambassador",transport="rest")
print(f)

Success:True message:
Request:
request {
  data {
    tensor {
      shape: 1
      shape: 1
      values: 0.2506330779508359
    }
  }
}
response {
  meta {
    puid: "hvi8q44f3jpmuu5tt8buntu68j"
    requestPath {
      key: "classifier"
      value: "seldonio/mock_classifier_rest:1.2"
    }
  }
  data {
    names: "proba"
    tensor {
      shape: 1
      shape: 1
      values: 0.06500763805294546
    }
  }
}
reward: 1.0

Response:



### gRPC

In [8]:
p = sc.predict(gateway="ambassador",transport="grpc")
f = sc.feedback(p.request,p.response,1.0,gateway="ambassador",transport="grpc")
print(f)

Success:True message:
Request:
request {
  data {
    tensor {
      shape: 1
      shape: 1
      values: 0.9346599059629088
    }
  }
}
response {
  meta {
    puid: "qmg06c106kep9bel6bim9mq66i"
    requestPath {
      key: "classifier"
      value: "seldonio/mock_classifier_rest:1.2"
    }
  }
  data {
    names: "proba"
    tensor {
      shape: 1
      shape: 1
      values: 0.12110507855414415
    }
  }
}
reward: 1.0

Response:



In [9]:
p = sc.predict(gateway="seldon",transport="grpc")
f = sc.feedback(p.request,p.response,1.0,gateway="seldon",transport="grpc")
print(f)

Success:True message:
Request:
request {
  data {
    tensor {
      shape: 1
      shape: 1
      values: 0.6573860138244305
    }
  }
}
response {
  meta {
    puid: "u9hmqtq62qn2etj8cs1avvfdu2"
    requestPath {
      key: "classifier"
      value: "seldonio/mock_classifier_rest:1.2"
    }
  }
  data {
    names: "proba"
    tensor {
      shape: 1
      shape: 1
      values: 0.09455193564436758
    }
  }
}
reward: 1.0

Response:



In [10]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [ ]:
!helm install ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8
!kubectl rollout status deploy/myabtest-abtest-df66c5c

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r.text)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r.text)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!helm delete myabtest --purge

## Serve Multi-Armed Bandit

In [ ]:
!helm install ../../helm-charts/seldon-mab --name mymab --set modela.image.name=seldonio/mock_classifier_rest:1.2 --set modelb.image.name=seldonio/mock_classifier_rest:1.2 --set oauth.key=oauth-key --set oauth.secret=oauth-secret

In [ ]:
!kubectl rollout status deploy/mymab-abtest-295a20e
!kubectl rollout status deploy/mymab-abtest-b8038b2
!kubectl rollout status deploy/mymab-abtest-5724b8a

### Get predictions

In [ ]:
sc = SeldonClient(deployment_name="mymab",namespace="seldon",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [ ]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

In [ ]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

In [ ]:
!helm delete mymab --purge